<a href="https://www.kaggle.com/code/shusrith/worker?scriptVersionId=173540278" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("/kaggle/input/mobifaller/train.csv")
x = df.drop("label", axis=1)
y = [1 if i == 0 else 0 for i in df["label"]]
x

,1,2,3,4,5,6,7,8,9,10,...,1191,1192,1193,1194,1195,1196,1197,1198,1199,1200
0,167.0,199.0,217.0,226.0,216.0,229.0,232.0,235.0,236.0,236.0,...,133.0,123.0,117.0,115.0,117.0,118.0,118.0,118.0,120.0,125.0
1,171.0,172.0,185.0,204.0,218.0,217.0,208.0,196.0,187.0,180.0,...,154.0,155.0,150.0,143.0,137.0,133.0,130.0,128.0,128.0,129.0
2,120.0,130.0,137.0,142.0,143.0,152.0,153.0,143.0,146.0,165.0,...,109.0,112.0,120.0,126.0,131.0,132.0,152.0,193.0,206.0,181.0
3,122.0,122.0,122.0,123.0,124.0,125.0,125.0,125.0,124.0,124.0,...,113.0,111.0,110.0,110.0,110.0,112.0,114.0,117.0,119.0,118.0
4,162.0,20.0,189.0,161.0,20.0,190.0,160.0,19.0,189.0,159.0,...,205.0,162.0,166.0,204.0,161.0,166.0,204.0,161.0,166.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,187.0,184.0,182.0,180.0,176.0,170.0,160.0,152.0,143.0,136.0,...,156.0,159.0,158.0,158.0,159.0,162.0,165.0,167.0,161.0,144.0
7996,173.0,172.0,171.0,170.0,170.0,171.0,171.0,171.0,171.0,170.0,...,95.0,89.0,85.0,83.0,86.0,88.0,92.0,101.0,114.0,127.0
7997,165.0,197.0,211.0,207.0,202.0,206.0,219.0,228.0,232.0,234.0,...,55.0,42.0,38.0,42.0,50.0,60.0,72.0,87.0,105.0,118.0
7998,128.0,128.0,128.0,128.0,128.0,128.0,128.0,129.0,129.0,129.0,...,114.0,116.0,121.0,122.0,117.0,103.0,96.0,89.0,85.0,87.0


In [2]:
from sklearn.model_selection import train_test_split

a, val_input, b, val_output = train_test_split(x, y, test_size=0.5, random_state=42)

In [3]:
from sklearn.neighbors import NearestNeighbors

def nearest_neighbour(neighbours,x):
    n=NearestNeighbors(n_neighbors=neighbours,algorithm="ball_tree")
    n.fit(x)
    dist,indices=n.kneighbors(x)
    return indices
    

In [4]:
len(x)

8000

In [5]:
import random
def smote(x,neighbours,n):
    k=nearest_neighbour(neighbours,x)
    l=[]
    for i in range(len(x)):
        for j in range(n):
            r=random.choice(k[i])
            dist=x[r]-x[i]
            rd=random.random()
            l.append(x[i] + (rd*dist))
    return l

In [6]:
y = pd.Series(y)
y.value_counts()

0    7021
1     979
Name: count, dtype: int64

In [7]:
enn=[]
for i,j in enumerate(y):
    if j==1:
        enn.append(x.iloc[i, :])
enn=np.array(enn)
        

In [8]:
enn.shape

(979, 1200)

In [9]:
l = smote(enn, 5, 3)
len(l)

2937

In [10]:
np.array(l).shape


(2937, 1200)

In [11]:
x=np.concatenate((l,a), axis=0)
x.shape

(6937, 1200)

In [12]:
y=np.concatenate(([1 for i in range(len(enn)*3)], b), axis=0)
y.shape

(6937,)

In [13]:
q = pd.DataFrame(x)
q["label"] = y
df = q.sample(frac=1)
x = df.drop("label", axis=1)
y = df["label"]
y.value_counts()

label
0    3508
1    3429
Name: count, dtype: int64

In [14]:
x = np.array(x).reshape((6937, 20, 20, 3))

In [15]:
y = np.array(y)

In [16]:
val_input=np.array(val_input).reshape((val_input.shape[0],20,20,3))

In [17]:
x.shape
val_output=np.array(val_output)

In [18]:
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,Dense,Flatten
from tensorflow.keras.models import Sequential

model=Sequential()
model.add(Input(shape=(20,20,3)))
model.add(Conv2D(32, (5,5),activation="relu",padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (5,5), activation="relu",padding='same' ))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))
model.summary()

2024-04-23 13:11:30.783813: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 13:11:30.783906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 13:11:30.945970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 20, 20, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       819,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 939,201 (3.58 MB)

 Trainable params: 939,201 (3.58 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, **kwargs):
        super(F1Score, self).__init__(**kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [20]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(), loss=tf.keras.losses.BinaryCrossentropy(), metrics=[F1Score(),"accuracy"])


In [21]:
model.fit(x,y,epochs=5,validation_data=(val_input,val_output))

Epoch 1/5
 54/217 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6028 - f1_score: 0.5581 - loss: 9.6582 

I0000 00:00:1713877911.090300      74 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713877911.106481      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7866 - f1_score: 0.7719 - loss: 3.7139

W0000 00:00:1713877918.294542      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


217/217 ━━━━━━━━━━━━━━━━━━━━ 17s 38ms/step - accuracy: 0.7872 - f1_score: 0.7725 - loss: 3.7014 - val_accuracy: 0.9967 - val_f1_score: 0.9866 - val_loss: 0.0118


W0000 00:00:1713877919.112541      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Epoch 2/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9830 - f1_score: 0.9829 - loss: 0.0560 - val_accuracy: 0.9977 - val_f1_score: 0.9908 - val_loss: 0.0079
Epoch 3/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9912 - f1_score: 0.9911 - loss: 0.0271 - val_accuracy: 0.9980 - val_f1_score: 0.9918 - val_loss: 0.0099
Epoch 4/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9932 - f1_score: 0.9932 - loss: 0.0270 - val_accuracy: 0.9965 - val_f1_score: 0.9858 - val_loss: 0.0121
Epoch 5/5
217/217 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9941 - f1_score: 0.9941 - loss: 0.0220 - val_accuracy: 0.9983 - val_f1_score: 0.9928 - val_loss: 0.0064
